In [1]:
import sys
 
sys.path.insert(0, "../")

In [2]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf
import torch

data = pd.read_csv(r"../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/graph_learning/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [3]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
max_Id = data["Id"].unique().max()
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)
data = pd.concat((data, gw_data))
data = data.sort_values("Id")
unique_ids = data["Id"].unique()
data = data.reset_index(drop=True)
data.pop("index")
data = data.sort_values(["Id", "Time"])
data = data.reset_index(drop=True)

In [4]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES
data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"
data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

In [6]:
## ADD counter column for relative position of each measurement
import numpy as np
uniques, counts = np.unique(data["Id"].values, return_counts=True)

counter = []
print(counts)
for count in counts:
    if count != 1:
        counter.extend((np.arange(count) / (count-1)))
    if count == 1:
        counter.extend([1])
data["counter"] = counter

[1 1 1 ... 6 5 2]


In [7]:
X_features = torch.tensor(data[FEATURES].values).type(torch.float32)
y = torch.tensor(data["Label"].values).type(torch.long)

In [8]:
import torch_geometric
from dataAnalysis.FeatureImportance import normalize

X_features = normalize(X_features)

In [9]:
import torch
edge_index = torch.tensor(pd.read_csv("dir_edge_index_sorted_ids_w_gw_corr.csv", header=None, skiprows=1).values.transpose(), dtype=torch.long)
rev_edge_index = torch.zeros_like(edge_index)
index = torch.LongTensor([1,0])
rev_edge_index[index] = edge_index

In [10]:
from torch_geometric.data import Data
import numpy as np

def ratio_bool_switch(tensor, ratio = 0.8):
    random = np.random.uniform(0, 1 ,tensor.shape[0])
    val_ratio_mask = (random >= ratio)
    train_ratio_mask = (random < ratio)
    val_mask = np.logical_and(tensor.tolist(), val_ratio_mask.tolist())
    train_mask = np.logical_and(tensor.tolist(), train_ratio_mask.tolist())
    return torch.from_numpy(train_mask).type(torch.bool), torch.from_numpy(val_mask).type(torch.bool)

train_mask_ser = data["Set"] != "Validation"
train_mask, val_mask = ratio_bool_switch(train_mask_ser.values)
test_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values, (data["Center"] == "Leipzig").values)).type(torch.bool)
test_gw_mask = torch.from_numpy(np.logical_and((data["Set"] == "Validation").values, (data["Center"] == "Greifswald").values)).type(torch.bool)

In [28]:
graph = Data(x=X_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=edge_index,
             test_gw_mask = test_gw_mask)
graph_rev = Data(x=X_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=rev_edge_index,
             test_gw_mask = test_gw_mask)

In [74]:
edge_index.shape

torch.Size([2, 7259211])

In [75]:
from torch_geometric.utils import to_undirected
graph = Data(x=X_features, train_mask = train_mask, test_mask=test_mask, val_mask=val_mask, y= y, edge_index=to_undirected(edge_index),
             test_gw_mask = test_gw_mask)


In [19]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        HEADS = 5
        
        conv_1= GATConv(input_dim, hidden_dim,heads=HEADS, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x, attention_w_1 = self.conv_1(x, edge_index, return_attention_weights = True)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x, attention_w_2 = self.conv_end(x, edge_index, return_attention_weights = True)
        return x, (attention_w_1, attention_w_2)
            
    def predict_proba(self, graph, mask):
        with torch.inference_mode():
            self.eval()
            logits = self.forward(graph)
            scores = torch.sigmoid(torch.squeeze(logits[0][mask]))
            scores = torch.unsqueeze(scores, 0)
            proba_predict = torch.concat((1- scores, scores), dim = 0)
            return torch.transpose(proba_predict, 0, 1)
            
    def predict(self, graph, mask, threshold = 0.5):
        return (self.predict_proba(graph, mask)[:, 1] >= threshold).type(torch.int)

In [23]:
model_rev = GraphNeuralNetwork().cpu() #.to(device)
model_rev.load_state_dict(torch.load("../models/rev_directed_gat_wo_pos.pt", map_location=torch.device('cpu')))
model = GraphNeuralNetwork().cpu() #.to(device)
model.load_state_dict(torch.load("../models/directed_gat_wo_pos.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [34]:
with torch.inference_mode():
    model.eval()
    logits, (attention_w) = model(graph)
    scores = torch.sigmoid(torch.squeeze(logits))
    scores = torch.unsqueeze(scores, 0)
    proba_predict = torch.concat((1- scores, scores), dim = 0)
    pred = torch.round(scores).squeeze()[test_mask]

In [33]:
with torch.inference_mode():
    model_rev.eval()
    logits, (attention_w) = model_rev(graph_rev)
    scores = torch.sigmoid(torch.squeeze(logits))
    scores = torch.unsqueeze(scores, 0)
    proba_predict = torch.concat((1- scores, scores), dim = 0)
    pred_rev = torch.round(scores).squeeze()[test_mask]

In [67]:
corr_pred_rev = (pred_rev == graph_rev.y[test_mask])
corr_pred = (pred == graph.y[test_mask])

rev_corr_pred = np.logical_and(corr_pred_rev == 1, corr_pred == 0)
rev_corr_pred.sum()


tensor(45598)

In [69]:
data[test_mask.numpy()][rev_corr_pred.numpy().astype(np.bool_)]#.describe()

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,...,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label,SexCategory,counter
1015074,583815,77,M,Control,Leipzig,Validation,AMB,1,0.0,NaN,...,NaN,8.4,97.3,NaN,199.0,4.01,8.8,0,0,1.000000
1015114,583848,56,M,Control,Leipzig,Validation,ED,1,900.0,NaN,...,186.42,9.3,90.0,NaN,434.0,4.62,14.8,0,0,0.000000
1015115,583848,56,M,Control,Leipzig,Validation,GEN,1,58080.0,NaN,...,184.32,8.1,89.4,NaN,363.0,4.05,13.9,0,0,0.200000
1015116,583848,56,M,Control,Leipzig,Validation,GEN,1,159720.0,NaN,...,203.36,8.0,92.3,NaN,329.0,4.01,10.1,0,0,0.400000
1015117,583848,56,M,Control,Leipzig,Validation,GEN,1,252660.0,NaN,...,142.23,7.9,94.7,NaN,291.0,3.98,12.1,0,0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381351,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,0.0,MICU,...,27.81,7.1,98.2,NaN,158.0,3.28,9.4,0,0,0.000000
1381352,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,267660.0,MICU,...,19.55,6.7,100.0,NaN,129.0,3.11,6.0,0,0,0.166667
1381353,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,354780.0,MICU,...,20.14,6.8,97.5,NaN,134.0,3.18,7.0,0,0,0.333333
1381354,778980,56,M,Sepsis,Leipzig,Validation,GEN,1,606720.0,MICU,...,17.88,6.7,96.6,NaN,121.0,3.20,7.2,0,0,0.500000


In [76]:
from dataAnalysis.Metrics import Evaluation
y_dict = Evaluation.create_y_dict(model_rev.predict(graph, test_mask), model_rev.predict_proba(graph, test_mask) ,graph.y[test_mask])
y_dict_gw = Evaluation.create_y_dict(model_rev.predict(graph, test_gw_mask), model_rev.predict_proba(graph, test_gw_mask) , graph.y[test_gw_mask])
print(Evaluation.get_df_metrics_from_pred(y_dict, y_dict_gw))

         Name       MCC  F1-Micro  F1-Macro  F1-Binary     AUROC     AUPRC
0     Leipzig  0.105392  0.962586  0.510562   0.040202  0.807568  0.048663
1  Greifswald  0.086959  0.970795  0.509065   0.032955  0.783786  0.042402


In [89]:
print(model_rev.state_dict()["conv_1.att_src"])
for key in model_rev.state_dict():
    print(key)
    print(model_rev.state_dict()[key].shape)#["conv_1.lin_src."]

tensor([[[ -5.5509,  -7.3917,   5.3115,   8.5988,  -7.0259,  -0.8020,  -6.6783,
           -5.9650,   5.9438,   2.5997,  -8.6165,  -5.2122,  -4.7542,  -7.2708,
           -7.5291,   7.0018,   5.1190,   6.5479,  -0.5558,  -0.6842,   2.2853,
           -0.6124,  -1.8540,  -6.4124,  -9.2630,  -7.4157,   4.0730,  -1.4049,
            5.5289,   5.7126,  -6.4629,  -6.9577,   5.0042,   3.6878,  -5.4086,
           -1.5344,   5.4401,  -5.2238,   5.2753,  -9.7615,   5.1172,   5.9925,
           -1.7588,   3.6685,   5.5855,   5.1395,  -6.3127,   6.0387,   6.0029,
            8.8247,   5.5432,   4.5888,  -8.5960,  -5.0837,   3.8377,  -1.8420,
           -5.1039,  -4.9215,   4.2555,  -0.4275,   3.4322,  -4.5593,   3.5347,
            5.5869,   5.5903,  -9.7874,  -8.3494,  -0.8473,   5.5489,   3.6597,
            3.6747,  -6.6002,   5.5651,  -7.3569,  -6.1538,   6.9242,   0.3783,
            5.3847,   5.1020,  -5.7713,   4.3229,  -9.6634,  -1.2521,   4.0553,
            3.8016,  -6.2739,  -6.5930, 

RuntimeError: Can't access the shape of an uninitialized parameter or buffer. This error usually happens in `load_state_dict` when trying to load an uninitialized parameter into an initialized one. Call `forward` to initialize the parameters before accessing their attributes.

In [31]:
edge_df = pd.DataFrame(graph.edge_index.t(), columns = ["source", "target"])
edge_df["attention_weights"] = attention_w[1][1]
for i in range(5):
    edge_df[f"attention_weights_{i}"] = attention_w[0][1][:, i]
edge_df[f"attention_weights_mean"] = torch.mean(attention_w[0][1], dim = 1)

In [32]:
edge_df

,source,target,attention_weights,attention_weights_0,attention_weights_1,attention_weights_2,attention_weights_3,attention_weights_4,attention_weights_mean
0,0,0,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
1,1,1,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
2,2,2,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
3,3,3,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
4,4,4,1.000000e+00,0.000000,0.000000,2.188420e-08,1.000000,1.000000e+00,0.400000
...,...,...,...,...,...,...,...,...,...
7259206,1819432,1819430,0.000000e+00,1.000000,1.000000,9.999998e-01,0.010834,5.173853e-11,0.602167
7259207,1819432,1819431,0.000000e+00,1.000000,1.000000,9.999999e-01,0.430858,2.381722e-05,0.686176
7259208,1819433,1819433,1.000000e+00,0.886244,0.000005,8.557699e-01,0.998996,6.175879e-09,0.548203
7259209,1819434,1819434,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000


In [73]:
edge_df["source_label"] = data["Label"].iloc[edge_df["source"]].reset_index(drop=True)
edge_df["target_label"] = data["Label"].iloc[edge_df["target"]].reset_index(drop=True)

edge_df["source_counter"] = data["counter"].iloc[edge_df["source"]].reset_index(drop=True)
edge_df["target_counter"] = data["counter"].iloc[edge_df["target"]].reset_index(drop=True)

edge_df["source_pred"] = torch.index_select(pred.squeeze(), 0, edge_index[0])
edge_df["target_pred"] = torch.index_select(pred.squeeze(), 0, edge_index[1])

In [74]:
middle_edge_df = edge_df[np.logical_and(edge_df["target_counter"] != 1, edge_df["target_counter"] != 0)]
middle_edge_df.sort_values("attention_weights")
## where are the attention heads low (0) or especeially high (-> 1)?

,source,target,attention_weights,attention_weights_0,attention_weights_1,attention_weights_2,attention_weights_3,attention_weights_4,attention_weights_mean,source_label,target_label,source_counter,target_counter,source_pred,target_pred
3633999,921597,921589,0.0,1.057076e-10,3.430381e-07,1.250125e-09,6.515525e-23,1.116182e-07,9.120244e-08,0,0,0.863636,0.681818,0.0,0.0
3938803,1006171,1006150,0.0,2.480298e-43,7.854222e-40,9.993143e-01,3.247005e-25,0.000000e+00,1.998629e-01,0,0,0.681818,0.363636,0.0,0.0
3938804,1006171,1006151,0.0,2.480298e-43,7.854222e-40,9.993143e-01,4.136962e-19,0.000000e+00,1.998629e-01,0,0,0.681818,0.378788,0.0,0.0
3938805,1006171,1006152,0.0,2.480298e-43,7.854222e-40,9.993143e-01,2.691979e-21,0.000000e+00,1.998629e-01,0,0,0.681818,0.393939,0.0,0.0
3938806,1006171,1006153,0.0,2.480298e-43,7.854222e-40,9.993143e-01,4.877231e-25,0.000000e+00,1.998629e-01,0,0,0.681818,0.409091,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198517,315568,315568,1.0,0.000000e+00,4.612282e-34,4.746074e-11,8.015552e-04,9.999999e-01,2.001603e-01,0,0,0.250000,0.250000,0.0,0.0
4631892,1166900,1166900,1.0,0.000000e+00,0.000000e+00,1.244817e-06,1.987549e-27,1.000000e+00,2.000002e-01,0,0,0.076923,0.076923,0.0,0.0
4631893,1166901,1166901,1.0,0.000000e+00,0.000000e+00,1.390931e-08,8.106259e-21,1.000000e+00,2.000000e-01,0,0,0.096154,0.096154,0.0,0.0
4631823,1166865,1166865,1.0,3.645202e-10,4.367634e-20,3.575913e-02,9.992600e-01,0.000000e+00,2.070038e-01,0,0,0.500000,0.500000,0.0,0.0


In [93]:
correct_target_edges = middle_edge_df[middle_edge_df["target_label"] == middle_edge_df["target_pred"]]
correct_target_edges_with_importance = correct_target_edges[correct_target_edges["attention_weights"] >= .2]
correct_target_edges_with_importance_wo_self_edges = correct_target_edges_with_importance[correct_target_edges_with_importance["source"] != correct_target_edges_with_importance["target"]]
correct_target_edges_with_importance_wo_self_edges[["source_counter", "target_counter"]].describe()

,source_counter,target_counter
count,207494.000000,207494.000000
mean,0.514939,0.377583
std,0.210556,0.216055
min,0.006098,0.003049
25%,0.350000,0.200000
50%,0.500000,0.333333
75%,0.666667,0.538329
max,1.000000,0.993902


In [102]:
from scipy.stats import pearsonr
pearsonr(correct_target_edges_with_importance_wo_self_edges["source_counter"],
         correct_target_edges_with_importance_wo_self_edges["attention_weights_mean"])

PearsonRResult(statistic=0.2563953847228662, pvalue=0.0)